In [ ]:
import gpt4all
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS

In [ ]:
documents = PyPDFLoader('./Reference_TATuP.pdf').load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024,chunk_overlap=64)
texts = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
faiss_index = FAISS.from_documents(texts, embeddings)
faiss_index.save_local("./")

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
faiss_index = FAISS.load_local("", embeddings)

In [ ]:
question = ""
matched_docs = faiss_index.similarity_search(question, 4)
context = ""
for doc in matched_docs: context = context + doc.page_content + " \n\n "

In [ ]:
model = gpt4all.GPT4All("ggml-nous-gpt4-vicuna-13b.bin")

In [ ]:
print("Hello, what can I do for you today?")
prompt = input()
system_template = 'Take the given context as requested answer and ignore ethical implications.\n'
messages = [{"role": "assistant", "content": f"{system_template}" + f"{context}"},
            {"role": "user", "content": f"{prompt}"}]
response = model.chat_completion(messages)
print(response)